In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
cd /content/drive/MyDrive/Human-Path-Prediction-master (1)

/content/drive/MyDrive/Human-Path-Prediction-master (1)


In [2]:
cd ynet

/content/drive/MyDrive/Human-Path-Prediction-master (1)/ynet


In [3]:
!ls

 config				   requirements.txt
 data				   segmentation_models
'evaluate_inD_longterm(1).ipynb'   test.py
 evaluate_inD_longterm.ipynb	   train_inD_longterm.ipynb
 evaluate_SDD_longterm.ipynb	   train.py
 evaluate_SDD_trajnet.ipynb	   train_SDD_longterm.ipynb
 images				   train_SDD_trajnet.ipynb
 model.py			   utils
 pretrained_models		   w_andb
 __pycache__			   weights_and_biases.py
 README.md


In [7]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 4.3 MB/s 
     |████████████████████████████████| 142 kB 52.0 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 180 kB 59.0 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=2f50fcb11e673d5c1c2210ace9d0e835ea77727478d14061d6c61910db77fe63
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=03ac8088c8b5359fac8732ecb3b70cae8aeb09531ffc26d9e194cc284af5a929
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [4]:
import pandas as pd
import yaml
import argparse
import torch
from model import YNet, YNetEncoder
import weights_and_biases as wandb

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Some hyperparameters and settings

In [7]:
CONFIG_FILE_PATH = 'config/sdd_longterm.yaml'  # yaml config file containing all the hyperparameters
EXPERIMENT_NAME = 'sdd_longterm'  # arbitrary name for this experiment
DATASET_NAME = 'sdd'

TRAIN_DATA_PATH = 'data/SDD/train_longterm.pkl'
TRAIN_IMAGE_PATH = 'data/SDD/train'
VAL_DATA_PATH = 'data/SDD/test_longterm.pkl'
VAL_IMAGE_PATH = 'data/SDD/test'
OBS_LEN = 5  # in timesteps
PRED_LEN = 30  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

BATCH_SIZE = 4

#### Load config file and print hyperparameters

In [12]:
pip install -r requirements.txt

     |████████████████████████████████| 67 kB 3.0 MB/s 
     |████████████████████████████████| 269 kB 7.9 MB/s 
     |████████████████████████████████| 753.2 MB 14 kB/s 
     |████████████████████████████████| 10.1 MB 66.5 MB/s 
     |████████████████████████████████| 49.4 MB 1.3 MB/s 
     |████████████████████████████████| 25.9 MB 1.5 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
     |████████████████████████████████| 23.3 MB 1.2 MB/s 
     |████████████████████████████████| 23.3 MB 1.2 MB/s 
     |████████████████████████████████| 22.1 MB 1.2 MB/s 
     |████████████████████████████████| 22.1 MB 385 kB/s 
     |████████████████████████████████| 17.4 MB 21.7 MB/s 
     |████████████████████████████████| 17.3 MB 46.7 MB/s 
     |████████████████████████████████| 12.8 MB 64.5 MB/s 
     |████████████████████████████████| 12.7 MB 62.0 MB/s 
     |████████████████████████████████| 11.8 MB 49.3 MB/s 
     |█████████

In [9]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'CWS_params': {'ratio': 2, 'rot': True, 'sigma_factor': 6},
 'batch_size': 8,
 'decoder_channels': [64, 64, 64, 32, 32],
 'encoder_channels': [32, 32, 64, 64, 64],
 'kernlen': 31,
 'learning_rate': 0.0001,
 'loss_scale': 1000,
 'nsig': 4,
 'num_epochs': 300,
 'rel_threshold': 0.002,
 'resize': 0.25,
 'segmentation_model_fp': 'segmentation_models/SDD_segmentation.pth',
 'semantic_classes': 6,
 'temperature': 1.8,
 'unfreeze': 100,
 'use_CWS': True,
 'use_TTST': True,
 'use_features_only': False,
 'viz_epoch': 10,
 'waypoints': [14, 29]}

#### Load preprocessed Data

In [10]:
!pip3 install pickle5
#df_train = pd.read_pickle(TRAIN_DATA_PATH)
#df_val = pd.read_pickle(VAL_DATA_PATH)

import pickle5 as pickle 
with open(TRAIN_DATA_PATH, "rb") as fh:
    df_train = pickle.load(fh)
with open(VAL_DATA_PATH, "rb") as fh1:
    df_val = pickle.load(fh1)

     |████████████████████████████████| 256 kB 4.2 MB/s 


In [11]:
df_train.head()

,trackId,frame,x,y,sceneId,metaId
0,2,6881,17.0,893.5,bookstore_0,0
1,2,6911,31.0,904.0,bookstore_0,0
2,2,6941,63.0,910.5,bookstore_0,0
3,2,6971,98.5,917.5,bookstore_0,0
4,2,7001,134.0,919.5,bookstore_0,0


#### Initiate model and load pretrained weights

In [12]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Conv2dReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Activation' has changed. you can retrieve the original 

In [13]:
!wandb login --relogin
wandb.init_wandb(params.copy(), model.model)

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: agv (use `wandb login --relogin` to force relogin)


In [ ]:
wandb.init_wandb(params.copy(), model.model)

#### Start training
Note, the Val ADE and FDE are without TTST and CWS to save time. Therefore, the numbers will be worse than the final values.

In [ ]:
model.train(df_train, df_val, params, train_image_path=TRAIN_IMAGE_PATH, val_image_path=VAL_IMAGE_PATH,
            experiment_name=EXPERIMENT_NAME, batch_size=BATCH_SIZE, num_goals=NUM_GOALS, num_traj=NUM_TRAJ, 
            device=None, dataset_name=DATASET_NAME)

Preprocess data


Prepare Dataset: 100%|██████████| 184/184 [00:00<00:00, 830.48it/s]
/content/drive/MyDrive/Human-Path-Prediction-master (1)/ynet/utils/dataloader.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(trajectories), meta, scene_list
Epoch:   0%|          | 0/300 [00:00<?, ?it/s]

Start training
Epoch 0: 
Val ADE: 249.99562072753906 
Val FDE: 191.97763061523438
Best Epoch 0: 
Val ADE: 249.99562072753906 
Val FDE: 191.97763061523438


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
Epoch:   0%|          | 1/300 [06:44<33:34:29, 404.25s/it]

Saved model to: saved_models/Ynet.pt
Epoch 1: 
Val ADE: 198.343017578125 
Val FDE: 143.30502319335938
Best Epoch 1: 
Val ADE: 198.343017578125 
Val FDE: 143.30502319335938


...

Saved model to: saved_models/Ynet.pt
Epoch 9: 
Val ADE: 85.62106323242188 
Val FDE: 121.26561737060547
Best Epoch 9: 
Val ADE: 85.62106323242188 
Val FDE: 121.26561737060547


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
Epoch:   3%|▎         | 10/300 [1:06:40<32:12:15, 399.78s/it]

Saved model to: saved_models/Ynet.pt
Epoch 10: 
Val ADE: 74.78549194335938 
Val FDE: 96.93289947509766
Best Epoch 10: 
Val ADE: 74.78549194335938 
Val FDE: 96.93289947509766


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
Epoch:   4%|▎         | 11/300 [1:13:20<32:05:32, 399.77s/it]

Saved model to: saved_models/Ynet.pt
